For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [22]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time, math
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [23]:
# Load API Credentials
with open('/Users/spart/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)

In [29]:
LOCATION = 'Colorado Springs, CO'
TERM = 'Chicken Wings'

In [30]:
JSON_FILE = "Data/Results_COS_ChickenWings.json"
JSON_FILE

'Data/Results_COS_ChickenWings.json'

In [31]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    if len(folder)>0:
        os.makedirs(folder,exist_ok=True)
        
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    with open(JSON_FILE,'w') as f:
        json.dump([],f) 
        
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Results_COS_ChickenWings.json not found. Saving empty list to file.


In [32]:
# Checking if they are results
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [33]:
results = yelp_api.search_query(location = LOCATION, term = TERM, offset = n_results)

results.keys()

dict_keys(['businesses', 'total', 'region'])

In [34]:
total_results = results['total']
total_results

304

In [35]:
results_per_page = len(results['businesses'])
results_per_page

20

In [36]:
n_pages = math.ceil((results['total'] - n_results) / results_per_page)
n_pages

16

In [37]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)

    n_results = len(previous_results)
   
    results = yelp_api.search_query(location = LOCATION,
                                    term = TERM, 
                                    offset = n_results)
    
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/16 [00:00<?, ?it/s]

In [38]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ytBUcB4xdnuGM5nDJir74Q,golden-flame-hot-wings-colorado-springs-2,Golden Flame Hot Wings,https://s3-media1.fl.yelpcdn.com/bphoto/o9WbE2...,False,https://www.yelp.com/biz/golden-flame-hot-wing...,196,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 38.9097629, 'longitude': -104.717...","[delivery, pickup]",$$,"{'address1': '5865 Stetson Hills Blvd', 'addre...",+17195709464,(719) 570-9464,5176.629893
1,Cbsnscv1pz1pPxLMda9AIA,angry-chicken-colorado-springs-2,Angry Chicken,https://s3-media1.fl.yelpcdn.com/bphoto/uJGERf...,False,https://www.yelp.com/biz/angry-chicken-colorad...,78,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 38.806996, 'longitude': -104.821783}","[delivery, pickup]",$$,"{'address1': '1869 S Nevada Ave', 'address2': ...",+17196949730,(719) 694-9730,9279.301392
2,n3elN1ApEXz5raVO9MQj7w,deluxe-wingz-colorado-springs-2,Deluxe Wingz,https://s3-media1.fl.yelpcdn.com/bphoto/LTWnSO...,False,https://www.yelp.com/biz/deluxe-wingz-colorado...,62,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 38.7957911001335, 'longitude': -1...","[delivery, pickup]",NaN,"{'address1': '3869 Astrozon Blvd', 'address2':...",+17193921158,(719) 392-1158,8787.600799
3,5kC28QlV8SbbFCDSdchY3A,wild-bills-buffalo-wings-colorado-springs-6,Wild Bill's Buffalo Wings,https://s3-media1.fl.yelpcdn.com/bphoto/ZpE2_9...,False,https://www.yelp.com/biz/wild-bills-buffalo-wi...,34,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",4.0,"{'latitude': 38.79650111877393, 'longitude': -...",[delivery],NaN,"{'address1': '3768 Astrozon Blvd', 'address2':...",+17193445062,(719) 344-5062,8710.976657
4,lGnCj90vz7xGUlvK_q6S8Q,the-dirty-byrd-colorado-springs-2,The Dirty Byrd,https://s3-media1.fl.yelpcdn.com/bphoto/e7aR3F...,False,https://www.yelp.com/biz/the-dirty-byrd-colora...,29,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"{'latitude': 38.83554, 'longitude': -104.82407}","[delivery, pickup]",NaN,"{'address1': '24 E Kiowa St', 'address2': None...",,,7247.559326


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
299,BAL4xOaBOtsoDDTAS-FWNg,lanna-thai-colorado-springs,Lanna Thai,https://s3-media4.fl.yelpcdn.com/bphoto/8d4NgP...,False,https://www.yelp.com/biz/lanna-thai-colorado-s...,172,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",3.5,"{'latitude': 38.9594841003418, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '8810 N Union Blvd', 'address2': ...",+17192820474,(719) 282-0474,9447.346016
300,ubDnfqz9VgGV7b6YDrG10A,china-cottage-fountain,China Cottage,https://s3-media2.fl.yelpcdn.com/bphoto/QwlEmq...,False,https://www.yelp.com/biz/china-cottage-fountai...,48,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,"{'latitude': 38.7326, 'longitude': -104.727752}",[delivery],$$,"{'address1': '7040 S US Highway 85-87', 'addre...",+17193900377,(719) 390-0377,16020.888223
301,-j-rDUUnlC5fu7M-1cs9AQ,marcos-pizza-briargate-colorado-springs-3,Marco's Pizza - Briargate,https://s3-media4.fl.yelpcdn.com/bphoto/T9PLLR...,False,https://www.yelp.com/biz/marcos-pizza-briargat...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 38.9675701, 'longitude': -104.78347}",[delivery],$,"{'address1': '9420 Briar Village Point', 'addr...",+17194343773,(719) 434-3773,10557.675775
302,g8jw7cDnEzOjFfx74qngyw,new-panda-colorado-springs,New Panda,https://s3-media1.fl.yelpcdn.com/bphoto/S5IKhi...,False,https://www.yelp.com/biz/new-panda-colorado-sp...,101,"[{'alias': 'chinese', 'title': 'Chinese'}]",2.5,"{'latitude': 38.7883579, 'longitude': -104.820...","[pickup, delivery]",$,"{'address1': '445 E Cheyenne Mountain Blvd', '...",+17195769211,(719) 576-9211,11057.073763
303,oRTZkXhREqXLSqeTJKsm6A,pizza-hut-fountain-6,Pizza Hut,https://s3-media3.fl.yelpcdn.com/bphoto/ZKJ6Ef...,False,https://www.yelp.com/biz/pizza-hut-fountain-6?...,34,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 38.7161475872586, 'longitude': -1...","[pickup, delivery]",$,"{'address1': '6859 Mesa Ridge Pkwy', 'address2...",+17193911200,(719) 391-1200,18329.558139


In [40]:
final_df.duplicated(subset='id').sum()

0

In [ ]:
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression = 'gzip',index = False)